# Using Hypothetical Document Embeddinga (HyDE) to Improve Retrieval

> 📚 This cookbook has an accompanying article with a complete walkthrough ["Optimizing Retrival with HyDE"](https://haystack.deepset.ai/blog/optimizing-retrieval-with-hyde)

In this coookbook, we are building Haystack components that allow us to easily incorporate HyDE into our RAG pipelines, to optimize retrieval.

> To learn more about HyDE and when it's useful, check out our [guide to Hypothetical Document Embeddings (HyDE)](https://docs.haystack.deepset.ai/v2.0/docs/hypothetical-document-embeddings-hyde)

## Install Requirements

In [1]:
!pip install haystack-ai sentence-transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In the following sections, we will be using the `OpenAIGenerator`, so we need to provide our API key 👇

In [19]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("Enter your openAI key:")

Enter your openAI key:··········


## Building a Pipeline for Hypothetical Document Embeddings

We will build a Haystack pipeline that generates 'fake' documents.
For this part, we are using the `OpenAIGenerator` with a `PromptBuilder` that instructs the model to generate paragraphs.

In [34]:
from haystack.components.generators.openai import OpenAIGenerator
from haystack.components.builders import PromptBuilder

generator = OpenAIGenerator(
    model="gpt-3.5-turbo",
    generation_kwargs={"n": 5, "temperature": 0.75, "max_tokens": 400},
)

template="""Given a question, generate a paragraph of text that answers the question.
            Question: {{question}}
            Paragraph:"""

prompt_builder = PromptBuilder(template=template)

Next, we use the `OutputAdapter` to transform the generated paragraphs into a List of Documents. This way, we will be able to use the `SentenceTransformersDocumentEmbedder` to create embeddings, since this component expects `List[Document]`

In [35]:
from haystack import Document
from haystack.components.converters import OutputAdapter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from typing import List

adapter = OutputAdapter(
    template="{{answers | build_doc}}",
    output_type=List[Document],
    custom_filters={"build_doc": lambda data: [Document(content=d) for d in data]}
)

embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
embedder.warm_up()

Finally, we create a custom component, `HypotheticalDocumentEmbedder`, that expects `documents` and can return a list of `hypotethetical_embeddings` which is the average of the embeddings from the "hypothetical" (fake) documents. To learn more about this technique and where it's useful, check out our [Guide to HyDE](https://docs.haystack.deepset.ai/v2.0/docs/hypothetical-document-embeddings-hyde)

In [36]:
from numpy import array, mean
from haystack import component

@component
class HypotheticalDocumentEmbedder:

  @component.output_types(hypothetical_embedding=List[float])
  def run(self, documents: List[Document]):
    stacked_embeddings = array([doc.embedding for doc in documents])
    avg_embeddings = mean(stacked_embeddings, axis=0)
    hyde_vector = avg_embeddings.reshape((1, len(avg_embeddings)))
    return {"hypothetical_embedding": hyde_vector[0].tolist()}

We add all of our components into a pipeline to genereate a hypothetical document embedding 🚀👇

In [37]:
from haystack import Pipeline

hyde = HypotheticalDocumentEmbedder()

pipeline = Pipeline()
pipeline.add_component(name="prompt_builder", instance=prompt_builder)
pipeline.add_component(name="generator", instance=generator)
pipeline.add_component(name="adapter", instance=adapter)
pipeline.add_component(name="embedder", instance=embedder)
pipeline.add_component(name="hyde", instance=hyde)

pipeline.connect("prompt_builder", "generator")
pipeline.connect("generator.replies", "adapter.answers")
pipeline.connect("adapter.output", "embedder.documents")
pipeline.connect("embedder.documents", "hyde.documents")

query = "What should I do if I have a fever?"
result = pipeline.run(data={"prompt_builder": {"question": query}})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
print(result["hyde"])

{'hypothetical_embedding': [0.003704594005830586, 0.04207931719720363, 0.040975296869874, 0.027549689263105394, 0.061635135859251025, 0.009835490654222668, 0.01163365535903722, 0.05085774138569832, 0.013786240434274077, -0.021979121296681114, -0.0708677776157856, -0.022855571750551463, 0.06218879371881485, 0.08835435658693314, 0.024315461330115796, -0.08007085993885994, 0.010663008620031177, 0.02036153059452772, 0.003111229557543993, -0.03922380032017827, -0.10432505160570145, -0.024043633975088595, -0.023948182072490453, 0.039578058663755654, -0.02324662944301963, 0.10965517312288284, 0.046073559671640396, 0.025174335693009198, -0.002544696070253849, 0.04209110364317894, 0.020509324688464402, -0.023811953701078893, 0.005540887874667532, -0.05318576171994209, -0.02349089856725186, 0.08110470101237297, -0.03771654982119799, -0.023771672509610653, -0.03994525820016861, 0.0372186578810215, 0.008177673630416393, 0.10352100282907487, 0.01880072634667158, -0.01892170701175928, 0.089331518113

## Build a HyDE Component That Encapsulates the Whole Logic

This section shows you how to create a `HypotheticalDocumentEmbedder` that instead, encapsulates the entire logic, and also allows us to provide the embedding model as an optional parameter.

This "mega" components does a few things:

- Allows the user to pick the LLM which generates the hypothetical documents
- Allows users to define how many documents should be created with `nr_completions`
- Allows users to define the embedding model they want to use to generate the HyDE embeddings.



In [40]:
from haystack import Pipeline, component, Document, default_to_dict, default_from_dict
from haystack.components.converters import OutputAdapter
from haystack.components.embedders.sentence_transformers_document_embedder import SentenceTransformersDocumentEmbedder
from haystack.components.generators.openai import OpenAIGenerator
from haystack.components.builders import PromptBuilder

from typing import Dict, Any, List
from numpy import array, mean
from haystack.utils import Secret


@component
class HypotheticalDocumentEmbedder:

    def __init__(
        self,
        instruct_llm: str = "gpt-3.5-turbo",
        instruct_llm_api_key: Secret = Secret.from_env_var("OPENAI_API_KEY"),
        nr_completions: int = 5,
        embedder_model: str = "sentence-transformers/all-MiniLM-L6-v2",
    ):
        self.instruct_llm = instruct_llm
        self.instruct_llm_api_key = instruct_llm_api_key
        self.nr_completions = nr_completions
        self.embedder_model = embedder_model
        self.generator = OpenAIGenerator(
            api_key=self.instruct_llm_api_key,
            model=self.instruct_llm,
            generation_kwargs={"n": self.nr_completions, "temperature": 0.75, "max_tokens": 400},
        )
        self.prompt_builder = PromptBuilder(
            template="""Given a question, generate a paragraph of text that answers the question.
            Question: {{question}}
            Paragraph:
            """
        )

        self.adapter = OutputAdapter(
            template="{{answers | build_doc}}",
            output_type=List[Document],
            custom_filters={"build_doc": lambda data: [Document(content=d) for d in data]},
        )

        self.embedder = SentenceTransformersDocumentEmbedder(model=embedder_model, progress_bar=False)
        self.embedder.warm_up()

        self.pipeline = Pipeline()
        self.pipeline.add_component(name="prompt_builder", instance=self.prompt_builder)
        self.pipeline.add_component(name="generator", instance=self.generator)
        self.pipeline.add_component(name="adapter", instance=self.adapter)
        self.pipeline.add_component(name="embedder", instance=self.embedder)
        self.pipeline.connect("prompt_builder", "generator")
        self.pipeline.connect("generator.replies", "adapter.answers")
        self.pipeline.connect("adapter.output", "embedder.documents")

    def to_dict(self) -> Dict[str, Any]:
        data = default_to_dict(
            self,
            instruct_llm=self.instruct_llm,
            instruct_llm_api_key=self.instruct_llm_api_key,
            nr_completions=self.nr_completions,
            embedder_model=self.embedder_model,
        )
        data["pipeline"] = self.pipeline.to_dict()
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> "HypotheticalDocumentEmbedder":
        hyde_obj = default_from_dict(cls, data)
        hyde_obj.pipeline = Pipeline.from_dict(data["pipeline"])
        return hyde_obj

    @component.output_types(hypothetical_embedding=List[float])
    def run(self, query: str):
        result = self.pipeline.run(data={"prompt_builder": {"question": query}})
        # return a single query vector embedding representing the average of the hypothetical document embeddings
        stacked_embeddings = array([doc.embedding for doc in result["embedder"]["documents"]])
        avg_embeddings = mean(stacked_embeddings, axis=0)
        hyde_vector = avg_embeddings.reshape((1, len(avg_embeddings)))
        return {"hypothetical_embedding": hyde_vector[0].tolist()}

## Use HyDE For Retrieval

Let's see how we can use this component in a full pipeline. First, let's index some documents into an `InMemoryDocumentStore`

In [44]:
from datasets import load_dataset, Dataset

from haystack import Pipeline, Document
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore

embedder_model = "sentence-transformers/all-MiniLM-L6-v2"


def index_docs(data: Dataset):
    # create a data store and indexing pipeline with the components
    document_store = InMemoryDocumentStore()

    pipeline = Pipeline()
    pipeline.add_component("cleaner", DocumentCleaner())
    pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=10))
    pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model=embedder_model))
    pipeline.add_component("writer", DocumentWriter(document_store=document_store, policy="skip"))

    # connect the components
    pipeline.connect("cleaner", "splitter")
    pipeline.connect("splitter", "embedder")
    pipeline.connect("embedder", "writer")

    # index the documents and return the data store
    pipeline.run({"cleaner": {"documents": [Document.from_dict(doc) for doc in data["train"]]}})
    return document_store


data = load_dataset("Tuana/game-of-thrones")
doc_store = index_docs(data)

Generating train split:   0%|          | 0/2357 [00:00<?, ? examples/s]

Batches:   0%|          | 0/103 [00:00<?, ?it/s]

We can now run a retrieval pipeline that doesn't just retrieve based on the query embeddings, instead, it uses the `HypotheticalDocumentEmbedder` to create hypothetical document embeddings based on our `query` and uses these new embeddings to retrieve documents.

In [45]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

def retriever_with_hyde(doc_store):
    hyde = HypotheticalDocumentEmbedder(instruct_llm="gpt-3.5-turbo", nr_completions=5)
    retriever = InMemoryEmbeddingRetriever(document_store=doc_store)

    retrieval_pipeline = Pipeline()
    retrieval_pipeline.add_component(instance=hyde, name="query_embedder")
    retrieval_pipeline.add_component(instance=retriever, name="retriever")

    retrieval_pipeline.connect("query_embedder.hypothetical_embedding", "retriever.query_embedding")
    return retrieval_pipeline

retrieval_pipeline = retriever_with_hyde(doc_store)
query = "Who is Araya Stark?"
retrieval_pipeline.run(data={"query_embedder": {"query": query}, "retriever": {"top_k": 5}})

{'retriever': {'documents': [Document(id=f21283f81311acf8c402cc0ef79a1fcc1d583fdfade0f48939c08d2242a44985, content: ''''Arya Stark''' is a fictional character in American author George R. R. Martin's ''A Song of Ice a...', meta: {'name': '43_Arya_Stark.txt', 'source_id': '557ad8704d47b0e12aef859563c87c86782b832f3d06909ede1c0e36dc039540'}, score: 0.7332133276298038),
   Document(id=a89c0e8f7acd10f8e7b87577f00287f4f82d78e316e2c32f25ffedaafaae8f10, content: '===Arya Stark===
   '''Arya Stark''' portrayed by Maisie Williams. Arya Stark of House Stark is the you...', meta: {'name': '349_List_of_Game_of_Thrones_characters.txt', 'source_id': '4c711b50138e5db004b183dd8e5e7a3fe3bf7360b317d61f0a31168fbd311c1e'}, score: 0.6954960060672856),
   Document(id=097bcaeffac04eaa9f97320bb9c15a692e95a427a86f60270abb18c079ebd867, content: '=== Sansa Stark ===
   Sansa Stark is the second child and elder daughter of Eddard and Catelyn Stark. ...', meta: {'name': '30_List_of_A_Song_of_Ice_and_Fire_characters